# Property classification

This notebook illustrates the capability of AEON.py to classify BN models with respect to dynamic properties. It also shows how to load and generate "arbitrary" classification archives.

In particular, we first demonstrate in detail how to classify the behaviour of a BN model based on HCTL properties, and then we briefly show other methods that can create different "classification archives". Note that all such archives can be explored using the [BN Classifier](https://github.com/sybila/biodivine-bn-classifier) user iterface.

This notebook assumes that you already have a basic understanding of AEON.py as presented in the remaining notebooks in this folder. However, you should be able to follow this tutorial even if you have not read the remaining notebooks, just keep in mind that you can find more information about the presented features there.

In [1]:
from biodivine_aeon import *
from pathlib import Path

### HCTL classification

#### Exploring input format for HCTL classification

We first demonstrate how the input for the HCTL classification looks like.

It is essentially a combination of:
- A partially specified Boolean network in the AEON format.
- Dynamic assertions (HCTL formulae) which restrict the set of valid model concretizations.
- Dynamic properties (HCTL formulae) that are used to classify the valid models.

The assertions/properties are given as model annotations. Let's explore how this is done.

In [2]:
raw_model_file = open("mapk-annotated.aeon", "r")
aeon_string = raw_model_file.read()
print(aeon_string)

# Note that some of the following properties have (slightly) simpler but equivalent formulas.
# However, we use these formulas to demonstrate various features of HCTL.


### Assertions

# There exists a state that is (a) in an attractor, and (b) with at least one phenotype variable active.
#! dynamic_assertion: #`3{x}: @{x}: ((!{y}: AG EF {y}) & (Apoptosis | Growth_Arrest | Proliferation))`#


### Properties

# The only attractors are fixed points with active `Apoptosis` (system always converges
# to the programmed cell death).
#! dynamic_property: p1: #`V{x}: @{x}: ((!{y}: AG EF {y}) => ((AX {x}) & Apoptosis))`#

# `Growth_arrest` is active and stable in some attractor.
#! dynamic_property: p2: #`3{x}: @{x}: ((!{y}: AG EF {y}) & Growth_Arrest & ~(EF ~Growth_Arrest))`#

# `Proliferation` is active and stable in some attractor.
#! dynamic_property: p3: #`3{x}: @{x}: ((!{y}: AG EF {y}) & Proliferation & ~(EF ~Proliferation))`#

# `Proliferation` and `Apoptosis` are simultaneously active 

We can automatically read the assertions/properties from the model file annotations and list them.

In [3]:
annotations = ModelAnnotation.from_model_string(aeon_string)
assertions = get_model_assertions(annotations)
named_properties = get_model_properties(annotations)

for assertion in assertions:
    print("Assertion:\t", assertion)
print()
for (name, prop) in named_properties:
    print(f"Property named `{name}`:\t", prop)

Assertion:	 3{x}: @{x}: ((!{y}: AG EF {y}) & (Apoptosis | Growth_Arrest | Proliferation))

Property named `p1`:	 V{x}: @{x}: ((!{y}: AG EF {y}) => ((AX {x}) & Apoptosis))
Property named `p2`:	 3{x}: @{x}: ((!{y}: AG EF {y}) & Growth_Arrest & ~(EF ~Growth_Arrest))
Property named `p3`:	 3{x}: @{x}: ((!{y}: AG EF {y}) & Proliferation & ~(EF ~Proliferation))
Property named `p4`:	 3{x}: @{x}: ((!{y}: AG EF {y}) & Apoptosis & Proliferation)


We see that assertions are just simple HCTL formulae (in our case a single formula). On the other hand, properties also have names, which are used in post-processing of the classification results. 

Notebook `model_checking.ipynb` provides further information on HCTL properties. For detailed HCTL syntax, refer to [this Github page](https://github.com/sybila/biodivine-hctl-model-checker).

#### Running the HCTL classification procedure

The classifier categorizes the concrete BNs (parametrizations of the PSBN or colors) to classes based on properties they satisfy. First, only the colors satisfying all assertions are computed. The set of remaining colors is decomposed into categories, where each class contains colors that (universally) satisfy the same set of properties.

It is however fairly straightforward to run the classification. You just provide a path to the annotated model file, and a path for the result bundle, everything else is handled internally.

The result is a zip archive with:
- A report of the whole classification procedure.
- Raw BDD files encoding each non-empty class.
- The original annotated aeon model input (to reconstruct the results if needed).

In [4]:
model_path = "mapk-annotated.aeon"
output_zip = "classification-archive.zip"
run_hctl_classification(model_path, output_zip)

Loaded model and properties out of `mapk-annotated.aeon`.
Parsing formulae and generating symbolic representation...
Successfully parsed all 1 required property (assertion) and all 4 classification properties.
Successfully encoded model with 18 variables and 4 parameters.
Model admits 16 instances.
Evaluating required properties (this may take some time)...
Required properties successfully evaluated.
15 instances satisfy all required properties.
Evaluating classification properties (this may take some time)...
Classification properties successfully evaluated.
Generating classification mapping based on model-checking results...
Results saved to `classification-archive.zip`.


We recommend loading the output into our interactive [GUI explorer](https://github.com/sybila/biodivine-bn-classifier).
However, you can explore the archive manually, or analyze it here in Python as well (see `load_class_archive` below).

We can load the dictionary with all categories—a map of `string -> color set` pairs. The string "key" represents a binary encoding of which properties are satisfied by the particular class (in this encoding, properties are ordered alphabetically).

In [5]:
class_mapping, original_model_str = load_class_archive(output_zip)
print(class_mapping)

{'0001': ColorSet(cardinality=3), '0100': ColorSet(cardinality=3), '1100': ColorSet(cardinality=9)}


Here, for example:
- 9 colors satisfy properties `p1` and `p2` simulataneously.
- 3 colors satisfy only `p2`
- 3 colors satisfy only `p4`
- No color satisfies property `p3`

### Attractor-based classification

Another method that can generate a "classification archive" is attractor-based classification. Here, we are concerned with the long-term behaviour of the possible network attractors (i.e. stability, oscillation, or disorder), and the absolute number of such attractors. Note that this is mostly equivalent to the analysis workflow originally presented in the online version of [AEON](https://biodivine.fi.muni.cz/aeon/v0.4.0/index.html).

Here, we don't need to specify any special properties or annotations. However, for convenience, we use the same annotated model as in our previous example.

In [6]:
run_attractor_classification(model_path, "attractor-archive.zip")

We can work with the result of this process in the same way we worked with the HCTL classification result. In particular, you can also visualize this classification using the interactive GUI. Notice that in this case, the class labels are actually the types of attractors encountered in each class, as opposed to HCTL properties.

In [7]:
attr_classes, original_model_str = load_class_archive("attractor-archive.zip")
print(attr_classes)

{'1 x Stability': ColorSet(cardinality=8), '1 x Disorder': ColorSet(cardinality=6), '2 x Stability': ColorSet(cardinality=2)}


Specifically, we have that:
 - 2 colors admit bistable behaviour with two steady states.
 - 8 colors admit a single steady state.
 - 6 colors admit a single complex disordered attractor.

#### Custom classification archives

Finally, we can also generate classification archives based on completely arbitrary colored sets. However, it is recommended that the sets should be at least pair-wise disjoint, otherwise the interpretation of some of the post-processing results can be complicated. 

In the following example, we simply separate the colors based on the value of two network input variables:

In [8]:
stg = SymbolicAsyncGraph(BooleanNetwork.from_aeon(original_model_str))
dna_on = stg.fix_implicit_function("DNA_damage", [], True)
dna_off = stg.fix_implicit_function("DNA_damage", [], False)
egfr_on = stg.fix_implicit_function("EGFR_stimulus", [], True)
egfr_off = stg.fix_implicit_function("EGFR_stimulus", [], False)

class_1 = dna_on.intersect(egfr_on)
class_2 = dna_on.intersect(egfr_off)
class_3 = dna_off

custom_mapping = {
    "DNA_with_EGFR": class_1,
    "DNA_without_EGFR": class_2,
    "no_DNA": class_3
}

save_class_archive("custom-archive.zip", custom_mapping, stg.network().to_aeon())

If we load the resulting archive into our GUI explorer, we can easily visualise the classification as a tree with two decisions.